# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [2]:
bad = open('../data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('../data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [4]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [28]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


In [8]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [23]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('естественно', 'естесственно'), 17),
 (('вообще', 'ваще'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [9]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [10]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [15]:
# создайте множество, чтобы проверять вхождения
vocab = set([i for j in corpus for i in j.split()])

## ваш код здесь

In [16]:
len(vocab)

137738

In [28]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    if word in vocab:
        return 0
    else:
        return 1
    ## ваш код здесь
    
    

In [29]:
predict_mistaken('опофеоз', vocab)

1

In [30]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0 
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка 
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

pairs = []

for sent1, sent2 in zip(true, bad):
    pairs += align_words(sent1, sent2)

## ваш код здесь
for right, wrong in pairs:
    if right == wrong:
        y_true.append(0)
    else:
        y_true.append(1)
    y_pred.append(predict_mistaken(wrong, vocab))

In [31]:
pairs[:10]

[('симпатичнейшее', 'симпатичнейшое'),
 ('шпионское', 'шпионское'),
 ('устройство', 'устройство'),
 ('такой', 'такой'),
 ('себе', 'себе'),
 ('гламурный', 'гламурный'),
 ('фотоаппарат', 'фотоаппарат'),
 ('девушки', 'девушки'),
 ('бонда', 'бонда'),
 ('миниатюрная', 'миниатюрная')]

In [32]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      8707
          1       0.55      0.88      0.68      1305

avg / total       0.92      0.89      0.90     10012



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [5]:
corpus = [sent.split() for sent in open('../data/corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [8]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [6]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [7]:
P('апофеоз')

1.8025455548325345e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [8]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [12]:
%%time
correction('опефеоз')

Wall time: 279 ms


'апофеоз'

Давайте поподробнее разберем, что происходит в функции edits.

In [13]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [14]:
splits[:10]

[('', 'опефеоз'),
 ('о', 'пефеоз'),
 ('оп', 'ефеоз'),
 ('опе', 'феоз'),
 ('опеф', 'еоз'),
 ('опефе', 'оз'),
 ('опефео', 'з'),
 ('опефеоз', '')]

In [15]:
deletes = [L + R[1:] for L, R in splits if R]

In [16]:
deletes[:10]

['пефеоз', 'оефеоз', 'опфеоз', 'опееоз', 'опефоз', 'опефез', 'опефео']

In [17]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [18]:
transposes[:10]

['поефеоз', 'оепфеоз', 'опфееоз', 'опеефоз', 'опефоез', 'опефезо']

In [19]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [20]:
len(replaces)

231

In [21]:
inserts = [L + c + R for L, R in splits for c in letters]

In [22]:
inserts[:10]

['йопефеоз',
 'цопефеоз',
 'уопефеоз',
 'копефеоз',
 'еопефеоз',
 'нопефеоз',
 'гопефеоз',
 'шопефеоз',
 'щопефеоз',
 'зопефеоз']

Для оценки используем просто долю правильных исправлений.

In [25]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.7333333333333333
10
0.8384615384615385
20
0.8441558441558441
30
0.8434343434343434
40
0.847953216374269
50
0.8518518518518519
60
0.8444444444444444
70
0.849009900990099
80
0.8490967056323061
90
0.8516699410609038
100
0.850358422939068
110
0.853904282115869
120
0.8538283062645011
130
0.8527472527472527
140
0.8516746411483254
150
0.8574144486692015
160
0.8567216981132075
170
0.8562605277933745
180
0.8536324786324786
190
0.8547094188376754
200
0.8576238576238576
210
0.8588929219600726
220
0.8594687232219366
230
0.8595482546201232
240
0.8635485117897178
250
0.8615497612926919
260
0.862469222652128
270
0.8600405679513184
280
0.8602775088738303


KeyboardInterrupt: 

In [43]:
print(correct/total)

0.860593220338983


In [45]:
%%time
correction('нав')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 202 µs


'на'

In [46]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 2.06 s, sys: 28 ms, total: 2.08 s
Wall time: 2.09 s


'насмехатьсяаававттававаываываы'

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [29]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [3]:
# в питоне есть библиотеке для нахождения близких строк

In [9]:
from difflib import get_close_matches

In [10]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

Wall time: 962 ms


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [11]:
import textdistance


In [12]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [13]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

Wall time: 4.24 s


('апофеоз', 0.7142857142857143)

In [14]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

Wall time: 1min 15s


('апофеоз', 0.8571428571428572)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [15]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [17]:
corpus = [sent.split() for sent in open('../data/corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [18]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [19]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [21]:
%%time
get_closest_match_vec('опофеоз', X, vec)

Wall time: 111 ms


['апофеоз', 'апофеозом', 'апофеоза']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [25]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    # ваш код здесь
    closest_matches = get_closest_match_vec(text, X, vec, TOPN=10)
    closest = get_closest_match_with_metric(text, closest_matches)[0]
    return closest

In [26]:
get_closest_hybrid_match('алкогнль', X, vec)

'алкоголь'

In [42]:
# оцените качество также как и раньше (если будет долго работать возьмите кусок данных)
# посмотрите на ошибки
def test_checker(checker, corpus_limit=None):
    #print('\t'.join(['mistake','predicted','true']))
    impropers = []
    correct = 0
    total = 0
    
    if not corpus_limit:
        corpus_limit = len(true)

    for i in range(corpus_limit):
        word_pairs = align_words(true[i], bad[i])

        for pair in word_pairs:

            predicted = checker(pair[1], X, vec)
            if predicted == pair[0]:
                correct += 1
            else:
                impropers.append((pair[1], predicted, pair[0]))
            total += 1
    
    return correct/total, impropers

Так как функция долго работает, возьмём первую сотню текстов:

In [44]:
%%time
score, improperly_corrected = test_checker(get_closest_hybrid_match, corpus_limit=100)

Wall time: 2min 29s


In [45]:
score

0.8172817281728173

In [66]:
len(improperly_corrected)

203

В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/

Попробуем использовать pymorphy2, чтобы улучшить качество:

In [48]:
import pymorphy2

In [71]:
def same_tag(taglist, tags):
    for tag in taglist:
        if tag.POS in tags:
            return True
    return False

def get_closest_hybrid_match_with_morpho(text, X, vec, metric=textdistance.levenshtein):
    # ваш код здесь
    analyzer = pymorphy2.analyzer.MorphAnalyzer()
    ##
    pos_tags = [i.POS for i in analyzer.tag(text)]
    ## оставляем только те слова, которые тэггятся теми же тегами
    closest_matches = [i for i in get_closest_match_vec(text, X, vec, TOPN=10) if same_tag(analyzer.tag(i), pos_tags)]
    if closest_matches:
        closest = get_closest_match_with_metric(text, closest_matches)[0]
    else:
        closest = text
    return closest

In [69]:
%%time
score, improperly_corrected = test_checker(get_closest_hybrid_match_with_morpho, corpus_limit=100)

Wall time: 3min 43s


In [70]:
score

0.8145814581458146

In [72]:
improperly_corrected

[('симпатичнейшое', 'пластичнейшими', 'симпатичнейшее'),
 ('шпионское', 'шпионские', 'шпионское'),
 ('гламурный', 'лагерный', 'гламурный'),
 ('бонда', 'банда', 'бонда'),
 ('superheadz', 'super', 'superheadz'),
 ('clap', 'place', 'clap'),
 ('camera', 'caterham', 'camera'),
 ('полчатся', 'ополчатся', 'получатся'),
 ('язычки', 'язычка', 'язычки'),
 ('оччччень', 'чечни', 'очень'),
 ('милые', 'милым', 'милые'),
 ('нащщот', 'защищено', 'насчет'),
 ('чавеса', 'чавес', 'чавеса'),
 ('попавшим', 'пропавшим', 'попавшим'),
 ('аварийно-спасательных',
  'аварийно-восстановительных',
  'аварийно-спасательных'),
 ('вобщем', 'общем', 'в'),
 ('как', 'как', 'общем'),
 ('вы', 'вы', 'как'),
 ('знаете', 'знаете', 'вы'),
 ('из', 'из', 'знаете'),
 ('моего', 'моего', 'из'),
 ('не', 'не', 'моего'),
 ('давнего', 'давнего', 'недавнего'),
 ('пропажу', 'продажу', 'пропажу'),
 ('почте.ру', 'лента.ру', 'почте.ру'),
 ('хороше', 'шорох', 'хорошо'),
 ('рите', 'триесте', 'рите'),
 ('патаму', 'аппарату', 'потому'),
 ('шта